In [2]:
import tensorflow as tf

In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
import itertools

In [6]:
import os

In [7]:
import tempfile

In [8]:
import pickle

In [9]:
from sklearn import datasets

In [10]:
from IPython.display import clear_output

In [11]:
training_set_n = pd.read_csv('/Users/wisdom/Downloads/boston_train/boston_train.csv').values

In [12]:
test_set_n = pd.read_csv('/Users/wisdom/Downloads/boston_train/boston_test.csv').values

In [13]:
prediction_set_n = pd.read_csv('/Users/wisdom/Downloads/boston_train/boston_predict.csv').values

In [14]:
def prepare_data(df):
    X_train = df[:, :-3]
    y_train = df[:,-3]    
    return X_train, y_train	

In [15]:
X_train, y_train = prepare_data(training_set_n)

In [16]:
X_test, y_test = prepare_data(test_set_n)

In [17]:
X_predict = prediction_set_n[:, :-2]

In [18]:
print(X_predict)

[[3.35900e-02 7.50000e+01 2.95000e+00 4.28000e-01 7.02400e+00 1.58000e+01
  5.40110e+00 2.52000e+02 1.83000e+01]
 [5.09017e+00 0.00000e+00 1.81000e+01 7.13000e-01 6.29700e+00 9.18000e+01
  2.36820e+00 6.66000e+02 2.02000e+01]
 [1.26500e-01 2.50000e+01 5.13000e+00 4.53000e-01 6.76200e+00 4.34000e+01
  7.98090e+00 2.84000e+02 1.97000e+01]
 [5.51500e-02 3.30000e+01 2.18000e+00 4.72000e-01 7.23600e+00 4.11000e+01
  4.02200e+00 2.22000e+02 1.84000e+01]
 [8.15174e+00 0.00000e+00 1.81000e+01 7.00000e-01 5.39000e+00 9.89000e+01
  1.72810e+00 6.66000e+02 2.02000e+01]
 [2.45220e-01 0.00000e+00 9.90000e+00 5.44000e-01 5.78200e+00 7.17000e+01
  4.03170e+00 3.04000e+02 1.84000e+01]]


In [19]:
print(X_train.shape, y_train.shape, X_predict.shape)

(400, 9) (400,) (6, 9)


In [20]:
feature_columns = [tf.feature_column.numeric_column('x', shape=X_train.shape[1:])]

In [21]:
print(feature_columns)

[NumericColumn(key='x', shape=(9,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [22]:
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns, model_dir="train")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8f8cb16f90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [23]:
def get_input_fn(data_df, label_df, num_epochs=None, batch_size=128, shuffle=True, steps=1000):
    def input_fn():
        ds = tf.data.Dataset.from_tensor_slices(({"x": data_df}, label_df)) if label_df is not None else tf.data.Dataset.from_tensor_slices(({"x": data_df}))
        if shuffle:
            ds = ds.shuffle(steps)
        ds = ds.batch(batch_size)
        return ds
    return input_fn     

In [24]:
train = estimator.train(input_fn=get_input_fn(X_train, y_train, shuffle=False))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
I

NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key linear/linear_model/x/weights not found in checkpoint
	 [[node save/RestoreV2 (defined at /Users/wisdom/opt/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]]

Original stack trace for 'save/RestoreV2':
  File "/Users/wisdom/opt/anaconda3/envs/tf/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/wisdom/opt/anaconda3/envs/tf/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/wisdom/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/wisdom/opt/anaconda3/envs/tf/lib/python3.7/site-packages/traitlets/config/application.py", 

In [ ]:
ev = estimator.evaluate(input_fn=get_input_fn(X_test, y_test, shuffle=False))
print(ev)

In [ ]:
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

In [ ]:
y = estimator.predict(input_fn=get_input_fn(X_predict, None, shuffle=False))

In [ ]:
pred_dicts = list(estimator.predict(input_fn=get_input_fn(X_predict, None, shuffle=False)))
clear_output()
for pred in pred_dicts:
    print(pred["predictions"][0])

In [ ]:
pred_dicts = list(y)
for pred in pred_dicts:
    print(pred["predictions"][0])

In [ ]:
tmpdir = tempfile.mkdtemp()
estimator_base_path = os.path.join(tmpdir, 'from_estimator')
feature_spec = tf.feature_column.make_parse_example_spec(feature_columns);
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec);
estimator_path = estimator.export_saved_model(estimator_base_path, export_input_fn)
imported = tf.saved_model.load(estimator_path)

In [ ]:
def predict(x):
    examples = []
    for row in x:
        print(row)
        feature = {}
        feature["x"] = tf.train.Feature(float_list=tf.train.FloatList(value=row))
        example = tf.train.Example(
            features=tf.train.Features(
                feature=feature
            )
        )
        examples.append(example.SerializeToString())
    return imported.signatures["predict"](
    examples=tf.constant(examples))

In [ ]:
print(predict(X_predict))